# Mongo tutorial

## Prerequisites

### Server status

Check the status of your Mongo cluster : `!sudo service mongod status`

If the cluster is inactive, `!sudo service mongod start` to start it.

In [1]:
# !sudo service mongodb status

### Import libraries

In [2]:
import datetime
from pprint import pprint

import pymongo
from pymongo import MongoClient

In [3]:
client = MongoClient('localhost', 27017)

In [4]:
# let's work in a test_database
db = client.test_database
posts = db.posts

In [5]:
post = {
    "author": "Mike",
    "text": "My first blog post!",
    "tags": ["mongodb", "python", "pymongo"],
    "date": datetime.datetime.utcnow()
}
post_id = posts.insert_one(post).inserted_id
post_id

ObjectId('6270760cc8043fcb12217711')

In [6]:
db.list_collection_names()

['posts', 'test']

In [7]:
pprint(posts.find_one())

{'_id': ObjectId('626fb40a9b31ddd39976fb83'),
 'author': 'Mike',
 'date': datetime.datetime(2022, 5, 2, 10, 35, 54, 238000),
 'tags': ['mongodb', 'python', 'pymongo'],
 'text': 'My first blog post!'}


You can launch a terminal aside, connect to your server with a mongo client and check that the value is present :

```bash
vagrant@nosql:~$ mongo
> show databases;
admin          0.000GB
config         0.000GB
local          0.000GB
test_database  0.000GB
> use test_database;
switched to db test_database
> db.posts.find()
{ 
    "_id" : ObjectId("..."), 
    "author" : "Mike", 
    "text" : "My first blog post!", 
    "tags" : [ "mongodb", "python", "pymongo"], 
    "date" : ISODate("2019-02-10T11:33:47.883Z") 
}
```

## I. Quick start

### First steps

**Q** : Create a document `{msg: 'hello'}` in the `test` collection with `insert_one()`. Fetch it back to display it. What is the `_id` for ?

NB : if the collection doesn't exist yet, MongoDB automatically creates it.

In [8]:
#db = client.test_database
tests = db.test
test = {
    "msg": "hello",
    "date": datetime.datetime.utcnow()
}
test_id = tests.insert_one(test).inserted_id
test_id

ObjectId('6270760cc8043fcb12217712')

**Q**: Display the number of documents inside the `test` collection

In [9]:
db.list_collection_names()
pprint(tests.find_one())

{'_id': ObjectId('626fb7cb9b31ddd39976fb86'),
 'date': datetime.datetime(2022, 5, 2, 10, 51, 55, 713000),
 'msg': 'hello'}


### Interacting with a database

We have 2 `.json` files we want to interact with inside the `data` folder. Let's first dump them into a `MovieLens` database, inside `users` and `movies` collections.

For this section, you will need to read a bit on [query operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors). Most methods on collections you will use have `filter` as a first parameter, on which you must pass a dictionary of query parameters.

**Q** : In the `MovieLens` database, load `data/movielens_movies.json` into `movies` and `data/movielens_users.json` into `users`. 

Use the dedicated shell command for this : `mongoimport --db <some_db> --collection <some_collection> --file <some_file>` 

In [10]:
!mongoimport --db MovieLens --collection movies --file ../data/json/movielens_movies.json

2022-05-02T19:23:40.346-0500	connected to: localhost
2022-05-02T19:23:40.408-0500	error inserting documents: multiple errors in bulk operation:
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 12 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 13 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 14 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 15 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 16 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 17 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 18 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 6 }
  - E11000 duplicate key error collection: MovieLens.movies index: _id_ dup key: { : 19 }
  - E11000 duplicate key error collection: Movi

In [11]:
!mongoimport --db MovieLens --collection users --file ../data/json/movielens_users.json

2022-05-02T19:23:40.744-0500	connected to: localhost
2022-05-02T19:23:41.112-0500	error inserting documents: multiple errors in bulk operation:
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6038 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6039 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6037 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6040 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6034 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6032 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6029 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6033 }
  - E11000 duplicate key error collection: MovieLens.users index: _id_ dup key: { : 6031 }
  - E11000 duplicate key error collec

**Q** : how many users are in the `MovieLens` database ?

In [12]:
db = client.MovieLens
users = db.users
users.count_documents({})

6040

**Q** : Display all comedies (the `genres` property equals `Comedy`). 

NB : You will need to find how to go through a `command_cursor`, then use the `pprint` function for a better display of those documents.

In [13]:
movies = db.movies
# movies.find()
for movie in movies.find():
    if movie["genres"] == "Comedy":
        pprint(movie)

{'_id': 5, 'genres': 'Comedy', 'title': 'Father of the Bride Part II (1995)'}
{'_id': 19,
 'genres': 'Comedy',
 'title': 'Ace Ventura: When Nature Calls (1995)'}
{'_id': 38, 'genres': 'Comedy', 'title': 'It Takes Two (1995)'}
{'_id': 52, 'genres': 'Comedy', 'title': 'Mighty Aphrodite (1995)'}
{'_id': 65, 'genres': 'Comedy', 'title': 'Bio-Dome (1996)'}
{'_id': 69, 'genres': 'Comedy', 'title': 'Friday (1995)'}
{'_id': 63,
 'genres': 'Comedy',
 'title': "Don't Be a Menace to South Central While Drinking Your Juice in the "
          'Hood (1996)'}
{'_id': 88, 'genres': 'Comedy', 'title': 'Black Sheep (1996)'}
{'_id': 109, 'genres': 'Comedy', 'title': 'Headless Body in Topless Bar (1995)'}
{'_id': 115, 'genres': 'Comedy', 'title': 'Happiness Is in the Field (1995)'}
{'_id': 119, 'genres': 'Comedy', 'title': 'Steal Big, Steal Little (1995)'}
{'_id': 96, 'genres': 'Comedy', 'title': 'In the Bleak Midwinter (1995)'}
{'_id': 101, 'genres': 'Comedy', 'title': 'Bottle Rocket (1996)'}
{'_id': 102

**Q** : Fetch and display the `name` and `occupation` for Clifford Johnathan. The second paramater for `find()` ([doc here](https://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.find)) is called the `projection` and is used to limit which data to fetch from the query.

In [14]:
users = db.users
users = users.find({"name": "Clifford Johnathan"}, {"occupation":1})
for user in users:
    print(user.get("occupation"))
#users

technician/engineer


**Q**: How many minors (by `age`) have rated movies ?

In [15]:
users = db.users.find({"age": {"$lt": 18}})
# print(len(users["movies"]))
#print(len(users))

#pprint(users)
cont=0
for user in users:
    cont+=1
    #pprint(user)
pprint(cont)
# “age”:{"$lt”:18}

222


**Q**: Display science fiction movies ('Sci-Fi') and suspense movies ('Thriller'). This time you need to use a regex to parse genres and look for those values.

In [16]:
users = db.users
users = users.collectionname.find({'genres':{'$regex':'*Sci-Fi|Thriller'}})
for user in users:
    pprint(user)

OperationFailure: Regular expression is invalid: nothing to repeat, full error: {'ok': 0.0, 'errmsg': 'Regular expression is invalid: nothing to repeat', 'code': 2, 'codeName': 'BadValue'}

**Q**: If we want more advanced textual search, we need a particular index. Use the `create_index()` method to index as [TEXT](https://docs.mongodb.com/manual/core/index-text/) the `genres` field of the `movies` collection.

In [ ]:
§

**Q**: Restart the search for science fiction and thriller movies with the operator `$text`

**Q**: Display the first 30 movies (`limit`) in alphabetical order (`sort`) by title

**Q**: How many users have seen the movie "Star Wars: Episode V - The Empire Strikes Back (1980)" (`_id 1196`) ? The `movies` argument is an array so we should try the [elemMatch](https://docs.mongodb.com/manual/reference/operator/projection/elemMatch/) operator here.

**Q**: And how many gave it a rating of 1 or 2 ?

### Updating data

**Q**: Insert a new user with the properties `name`, `gender` ('M' or'F'), `occupation` and `age`, using the `insert_one()` command. Display it with `find_one()`.

**Q**: Add an appreciation on a viewed movie with `update_one()`, add the movies property containing a table with a document (`movieid`, `rating`, `timestamp` with the value `datetime.datetime.utcnow()`).

You will need to read the documentation on [update operators](https://docs.mongodb.org/manual/reference/operator/update/).

**Q**: Find the number of users who have declared a `programmer` occupation. Modify them so that they are `developer`. Verify your update.

## II. Modelling a blog

We will now model a blog using Mongo. 

First, switch to a new `Blog` database. Each blog post will have the following arguments:

* The author (author field, string type)
* The date (date field, string type in YYYY-MM-DD format)
* The content (field content)
* Tags (field tags, a string array)
* A list of comments (field comments) containing:
 * The author (author field, string type)
 * The date (date field, string type in YYYY-MM-DD format)
 * The content (field content)


**Q**: Create a first post by `rick`, on January 15th, with the tags `mongodb` and `nosql`.

**Q**: Create a second post by `kate`, on January 21, with the tag `nosql` and a comment from `rick` on the same day.

**Q**: Display the author of the last post with the tag `nosql`

**Q**: Add a comment by `jack` on January 25, to `kate`'s post

**Q**: Display all comments by `kate`

## Postquisites

In [ ]:
!mongo test_database --eval 'db.dropDatabase()'

In [ ]:
!mongo MovieLens --eval 'db.dropDatabase()'

In [ ]:
!mongo Blog --eval 'db.dropDatabase()'